In [14]:
%%capture output 
%pip install boto3 pandas pyarrow python-dotenv h3 geopandas matplotlib shapely

In [15]:
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
import io
import h3
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point


In [16]:
load_dotenv('../wb_aws.env')

True

In [17]:
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_REGION')
)

s3_client = session.client('s3')

In [18]:
bucket_name = 'wbg-geography01'
folder_path = 'Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/'

In [27]:
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
for obj in response['Contents']:
    print(f"{obj['Key']} {obj['Size']} Bytes")


Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_0_2020.parquet 243987093 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_10_2020.parquet 242133942 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_15_2020.parquet 242227987 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_1_2020.parquet 242309771 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_20_2020.parquet 242344428 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_25_2020.parquet 242355331 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_30_2020.parquet 242410897 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_35_2020.parquet 242564785 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_40_2020.parquet 242742899 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_45_2020.parquet 242939383 Bytes
Space2Stats/parquet/GLOBAL/WorldPop_2020_Demographics/f_50_2020.parquet 243118293 Bytes
Space2Stats/parquet/GLOBAL/WorldPo

In [29]:
obj = s3_client.get_object(Bucket=bucket_name, Key=folder_path+'f_0_2020.parquet')
body = obj['Body'].read()

buffer = io.BytesIO(body)
df = pd.read_parquet(buffer)
df.head()

,SUM,MIN,MAX,MEAN,id
0,-1.0,-1.0,-1.0,-1.0,8600504efffffff
1,-1.0,-1.0,-1.0,-1.0,860172317ffffff
2,-1.0,-1.0,-1.0,-1.0,86001d0cfffffff
3,-1.0,-1.0,-1.0,-1.0,8600406b7ffffff
4,-1.0,-1.0,-1.0,-1.0,86005e6f7ffffff


In [30]:
filtered_df = df[df['SUM'] > 0]
filtered_df.head()

,SUM,MIN,MAX,MEAN,id
16,0.006808,5.537143e-05,0.000070,0.000061,860196107ffffff
17,0.004956,2.208996e-06,0.000045,0.000034,86019991fffffff
26,0.000330,4.243886e-07,0.000022,0.000014,8600a2817ffffff
32,0.008553,3.608915e-05,0.000058,0.000041,86015e7a7ffffff
43,0.003776,1.950261e-06,0.000048,0.000041,86015e827ffffff


In [31]:
def hex_to_point(h):
    lat, lng = h3.h3_to_geo(h)
    return Point(lng, lat)

In [35]:
df['geometry'] = df['id'].apply(hex_to_point)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

### Open Questions

- Should we rename attributes in dataset to `f'{aggregation_method}_{attribute_name}'` to keep the datasets explicit for the case when a dataset has multiple attributes to aggregate and avoid introspection of dataset name for subsequent processing steps?
- Should we use another convention `NaN`, or `null` instead of `-1` for no data values when numerical values? What about other types of values?
- Should we generate metadata for all the partitions of the each dataset, the eventual combined dataset with all partitions, or should we even wait until we have all the datasets integrated together?